<a href="https://colab.research.google.com/github/Abhi-v-b/LP-5/blob/main/HPC/Assignment%204/CUDA_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-6qiq_dft
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-6qiq_dft
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=4f55ebfd88b4ebba05d1c11e0d673415a7399915633263855371cf440ba7da93
  Stored in directory: /tmp/pip-ephem-wheel-cache-xv47vsqx/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cu
#include <iostream>
int main(){
  std::cout << "Hello World\n";
  return 0;
}


Hello World



Addition of two large vectors

In [14]:
%%cu

#include <iostream>
using namespace std;
__global__
void add(int* A, int* B, int* C, int size) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < size) {
C[tid] = A[tid] + B[tid];
}
}
void initialize(int* vector, int size) {
for (int i = 0; i < size; i++) {
vector[i] = rand() % 10;
}
}
void print(int* vector, int size) {
for (int i = 0; i < size; i++) {
cout << vector[i] << " ";
}
cout << endl;
}
int main() {
int N = 4;int* A, * B, * C;
int vectorSize = N;
size_t vectorBytes = vectorSize * sizeof(int);
A = new int[vectorSize];
B = new int[vectorSize];
C = new int[vectorSize];
initialize(A, vectorSize);
initialize(B, vectorSize);
cout << "Vector A: ";
print(A, N);
cout << "Vector B: ";
print(B, N);
int* X, * Y, * Z;
cudaMalloc(&X, vectorBytes);
cudaMalloc(&Y, vectorBytes);
cudaMalloc(&Z, vectorBytes);
cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);
int threadsPerBlock = 256;
int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
add<<<blocksPerGrid, threadsPerBlock>>>(X, Y, Z, N);
cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);
cout << "Addition: ";
print(C, N);
delete[] A;
delete[] B;
delete[] C;
cudaFree(X);
cudaFree(Y);
cudaFree(Z);
return 0;
}

/tmp/tmptr6zbazy/6d6a03d8-61b0-48e5-ada1-73a5ae753b8b.cu:1:2: error: invalid preprocessing directive #Addition
    1 | #Addition of Two Large Vectors
      |  ^~~~~~~~



Matrix Multiplication using CUDA C

In [12]:
%%cu
#include <iostream>
#include <cuda.h>
using namespace std;
#define BLOCK_SIZE 2
__global__ void gpuMM(float *A, float *B, float *C, int N)
{
// Matrix multiplication for NxN matrices C=A*B
// Each thread computes a single element of C
int row = blockIdx.y*blockDim.y + threadIdx.y;
int col = blockIdx.x*blockDim.x + threadIdx.x;
float sum = 0.f;for (int n = 0; n < N; ++n)
sum += A[row*N+n]*B[n*N+col];
C[row*N+col] = sum;
}
int main(int argc, char *argv[])
{int N;float K;
// Perform matrix multiplication C = A*B
// where A, B and C are NxN matrices
// Restricted to matrices where N = K*BLOCK_SIZE;
cout<<"Enter a Value for Size/2 of matrix";
cin>>K;
K = 1;
N = K*BLOCK_SIZE;
cout << "\n Executing Matrix Multiplcation" << endl;
cout << "\n Matrix size: " << N << "x" << N << endl;
// Allocate memory on the host
float *hA,*hB,*hC;
hA = new float[N*N];
hB = new float[N*N];
hC = new float[N*N];
// Initialize matrices on the host
for (int j=0; j<N; j++){
for (int i=0; i<N; i++){
hA[j*N+i] = 2;
hB[j*N+i] = 4;
}
}// Allocate memory on the device
int size = N*N*sizeof(float); // Size of the memory in bytes
float *dA,*dB,*dC;
cudaMalloc(&dA,size);
cudaMalloc(&dB,size);
cudaMalloc(&dC,size);
dim3 threadBlock(BLOCK_SIZE,BLOCK_SIZE);
dim3 grid(K,K);
cout<<"\n Input Matrix 1 \n";
for (int row=0; row<N; row++){
for (int col=0; col<N; col++){
cout<<hA[row*col]<<" ";
}
cout<<endl;
}
cout<<"\n Input Matrix 2 \n";
for (int row=0; row<N; row++){
for (int col=0; col<N; col++){
cout<<hB[row*col]<<" ";
}
cout<<endl;
}
// Copy matrices from the host to device
cudaMemcpy(dA,hA,size,cudaMemcpyHostToDevice);
cudaMemcpy(dB,hB,size,cudaMemcpyHostToDevice);
//Execute the matrix multiplication kernel
gpuMM<<<grid,threadBlock>>>(dA,dB,dC,N);// Now do the matrix multiplication on the CPU
/*float sum;
for (int row=0; row<N; row++){
for (int col=0; col<N; col++){
sum = 0.f;
for (int n=0; n<N; n++){
sum += hA[row*N+n]*hB[n*N+col];
}
hC[row*N+col] = sum;
cout << sum <<" ";
}
cout<<endl;
}*/
// Allocate memory to store the GPU answer on the host
float *C;
C = new float[N*N];
// Now copy the GPU result back to CPU
cudaMemcpy(C,dC,size,cudaMemcpyDeviceToHost);
// Check the result and make sure it is correct
cout <<"\n\n\n\n\n Resultant matrix\n\n";
for (int row=0; row<N; row++){
for (int col=0; col<N; col++){
cout<<C[row*col]<<" ";
}
cout<<endl;
}
cout << "Finished." << endl;
}

Enter a Value for Size/2 of matrix
 Executing Matrix Multiplcation

 Matrix size: 2x2

 Input Matrix 1 
2 2 
2 2 

 Input Matrix 2 
4 4 
4 4 





 Resultant matrix

16 16 
16 16 
Finished.

